In [1]:
#@title Data Exploration
data_name = "14lap" #@param ["14lap", "14res", "15res", "16res"]

import sys
sys.path.append("aste")
from data_utils import Data

path = f"aste/data/triplet_data/{data_name}/train.txt"
data = Data.load_from_full_path(path)

for s in data.sentences[:3]:
    print("tokens:", s.tokens)
    for t in s.triples:
        print("target:", (t.t_start, t.t_end))
        print("opinion:", (t.o_start, t.o_end))
        print("label:", t.label)
    print()

tokens: ['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.']
target: (16, 17)
opinion: (15, 15)
label: LabelEnum.positive

tokens: ['it', 'is', 'of', 'high', 'quality', ',', 'has', 'a', 'killer', 'GUI', ',', 'is', 'extremely', 'stable', ',', 'is', 'highly', 'expandable', ',', 'is', 'bundled', 'with', 'lots', 'of', 'very', 'good', 'applications', ',', 'is', 'easy', 'to', 'use', ',', 'and', 'is', 'absolutely', 'gorgeous', '.']
target: (4, 4)
opinion: (3, 3)
label: LabelEnum.positive
target: (9, 9)
opinion: (8, 8)
label: LabelEnum.positive
target: (26, 26)
opinion: (25, 25)
label: LabelEnum.positive
target: (31, 31)
opinion: (29, 29)
label: LabelEnum.positive

tokens: ['Easy', 'to', 'start', 'up', 'and', 'does', 'not', 'overheat', 'as', 'much', 'as', 'other', 'laptops', '.']
target: (2, 3)
opinion: (0, 0)
label: LabelEnum.positive



In [2]:
# Use pretrained SpanModel weights for prediction
import sys
sys.path.append("aste")
from pathlib import Path
from data_utils import Data, Sentence, SplitEnum
from wrapper import SpanModel

def predict_sentence(text: str, model: SpanModel) -> Sentence:
    path_in = "temp_in.txt"
    path_out = "temp_out.txt"
    sent = Sentence(tokens=text.split(), triples=[], pos=[], is_labeled=False, weight=1, id=0)
    data = Data(root=Path(), data_split=SplitEnum.test, sentences=[sent])
    data.save_to_path(path_in)
    model.predict(path_in, path_out)
    data = Data.load_from_full_path(path_out)
    return data.sentences[0]

text = "Did not enjoy the new Windows 8 and touchscreen functions ."
model = SpanModel(save_dir="pretrained_14lap", random_seed=0)
sent = predict_sentence(text, model)

for t in sent.triples:
    target = " ".join(sent.tokens[t.t_start:t.t_end+1])
    opinion = " ".join(sent.tokens[t.o_start:t.o_end+1])
    print()
    print(dict(target=target, opinion=opinion, sentiment=t.label))

{'command': 'cd /home/lid/MySpanASTE && allennlp predict pretrained_14lap/weights/model.tar.gz /home/lid/MySpanASTE/pretrained_14lap/temp_data/pred_in.json --predictor span_model --include-package span_model --use-dataset-reader  --output-file pretrained_14lap/temp_data/pred_out.json --cuda-device 0 --silent '}


2022-08-30 16:54:58,731 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-08-30 16:54:58,891 - INFO - allennlp.models.archival - loading archive file pretrained_14lap/weights/model.tar.gz
2022-08-30 16:54:58,892 - INFO - allennlp.models.archival - extracting archive file pretrained_14lap/weights/model.tar.gz to temp dir /tmp/tmp9jmid170
2022-08-30 16:55:03,578 - INFO - allennlp.common.params - dataset_reader.type = span_model
2022-08-30 16:55:03,578 - INFO - allennlp.common.params - dataset_reader.lazy = False
2022-08-30 16:55:03,578 - INFO - allennlp.common.params - dataset_reader.cache_directory = None
2022-08-30 16:55:03,578 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2022-08-30 16:55:03,578 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2022-08-30 16:55:03,578 - INFO - allennlp.common.params - dataset_reader.manual_multi_process_sharding = False
2022-08-30 16:55:03,578 - INFO - allennlp.common.pa

2022-08-30 16:55:21,224 - INFO - root - Loading a model trained before embedding extension was implemented; pass an explicit vocab namespace if you want to extend the vocabulary.
2022-08-30 16:55:21,371 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmp9jmid170
reading instances: 1it [00:00, 1239.09it/s]


################################################################################
################################################################################
{'locals': ('use_ner_embeds', False)}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_double_mix_embedder', False)}
{'locals': ('relation_head_type', 'proper')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys(['focal_loss_gamma', 'use_bi_affine_pruner', 'use_classify_mask_pruner', 'use_focal_loss', 'use_ner_scores_for_prune', 'use_ope_down_project', 'use_pair_feature_multiply', 'use_pairwise_down_project', 'use_span_loss_for_pruners', 'use_span_pair_aux_task', 'use_span_pair_aux_task_after_prune'])}
{'locals': {'self': ProperRelationExtractor(), 'make_feedforward': <function SpanModel.__init__.<locals>.make_feedforward at 0x7efdf47fe560>, 'span_emb_dim': 1556, 'feature_size': 20, 'spans_per_word': 0.5, 'positive_label_weight': 1.0, 'regularizer': None, 

In [4]:
random_seed = 0
path_train = f"aste/data/triplet_data/{data_name}/train.txt"
path_dev = f"aste/data/triplet_data/{data_name}/dev.txt"
path_test = f"aste/data/triplet_data/{data_name}/test.txt"
save_dir = f"outputs/{data_name}/seed_{random_seed}"

# Evaluate SpanModel F1 Score
import json

path_pred = "pred.txt"
model.predict(path_in=path_test, path_out=path_pred)
results = model.score(path_pred, path_test)
print(json.dumps(results, indent=2))

{'command': 'cd /home/lid/MySpanASTE && allennlp predict pretrained_14lap/weights/model.tar.gz /home/lid/MySpanASTE/pretrained_14lap/temp_data/pred_in.json --predictor span_model --include-package span_model --use-dataset-reader  --output-file pretrained_14lap/temp_data/pred_out.json --cuda-device 0 --silent '}


2022-08-30 16:59:20,341 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-08-30 16:59:20,482 - INFO - allennlp.models.archival - loading archive file pretrained_14lap/weights/model.tar.gz
2022-08-30 16:59:20,483 - INFO - allennlp.models.archival - extracting archive file pretrained_14lap/weights/model.tar.gz to temp dir /tmp/tmpvaeixb1q
2022-08-30 16:59:23,122 - INFO - allennlp.common.params - dataset_reader.type = span_model
2022-08-30 16:59:23,123 - INFO - allennlp.common.params - dataset_reader.lazy = False
2022-08-30 16:59:23,123 - INFO - allennlp.common.params - dataset_reader.cache_directory = None
2022-08-30 16:59:23,123 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2022-08-30 16:59:23,123 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2022-08-30 16:59:23,123 - INFO - allennlp.common.params - dataset_reader.manual_multi_process_sharding = False
2022-08-30 16:59:23,123 - INFO - allennlp.common.pa

2022-08-30 16:59:35,103 - INFO - root - Loading a model trained before embedding extension was implemented; pass an explicit vocab namespace if you want to extend the vocabulary.
2022-08-30 16:59:35,246 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmpvaeixb1q
reading instances: 328it [00:00, 2320.96it/s]


################################################################################
################################################################################
{'locals': ('use_ner_embeds', False)}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_double_mix_embedder', False)}
{'locals': ('relation_head_type', 'proper')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys(['focal_loss_gamma', 'use_bi_affine_pruner', 'use_classify_mask_pruner', 'use_focal_loss', 'use_ner_scores_for_prune', 'use_ope_down_project', 'use_pair_feature_multiply', 'use_pairwise_down_project', 'use_span_loss_for_pruners', 'use_span_pair_aux_task', 'use_span_pair_aux_task_after_prune'])}
{'locals': {'self': ProperRelationExtractor(), 'make_feedforward': <function SpanModel.__init__.<locals>.make_feedforward at 0x7ff81b573560>, 'span_emb_dim': 1556, 'feature_size': 20, 'spans_per_word': 0.5, 'positive_label_weight': 1.0, 'regularizer': None, 